In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn import metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import KFold
from model import ready_model
import matplotlib.pyplot as plt
from helper_functions import get_model_name

f:\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
X, Y = np.load("x.npy"), np.load("y.npy")

In [3]:
print(X.shape, Y.shape)

(1840, 1010) (1840,)


In [4]:
name = "crossval"
n_splits=10
kfold = KFold(n_splits=n_splits, shuffle=True)
accuracy = []
recall = []
fold_no = 1
for train, test  in kfold.split(X, Y):
    train_model = ready_model()
    train_model.compile(optimizer = Adam(learning_rate=0.0001), 
                        loss = SparseCategoricalCrossentropy(),
                        metrics = ['accuracy'])
    print(f'training fold number {fold_no}')

    save_dir = 'model/crossval/'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_no, name), 
            monitor='val_accuracy', verbose=1, 
            save_best_only=True, mode='max')

    callbacks_list = [checkpoint]

    history = train_model.fit(X[train], Y[train], epochs = 60, 
                            validation_data = (X[test], Y[test]), 
                            batch_size = 64,
                            callbacks = callbacks_list,
                            verbose = 0)


    train_model.load_weights(save_dir+get_model_name(fold_no, name))
    scores = train_model.evaluate(X[test], Y[test], verbose=0)
    accuracy.append(scores[1])
    y_pred = train_model.predict(X[test])
    y_preds_ready = y_pred.argmax(axis=1)
    recall.append(metrics.recall_score(Y[test], y_preds_ready, average='macro'))
    fold_no+=1
print("score per fold:")
for i in range(10):
    print(f'Accuracy in fold {i+1}: {accuracy[i]*100}, Recall in fold {i+1} : {recall[i]*100}')

training fold number 1

Epoch 00001: val_accuracy improved from -inf to 0.28804, saving model to model/crossval\model_crossval1.h5

Epoch 00002: val_accuracy improved from 0.28804 to 0.33152, saving model to model/crossval\model_crossval1.h5

Epoch 00003: val_accuracy improved from 0.33152 to 0.41304, saving model to model/crossval\model_crossval1.h5

Epoch 00004: val_accuracy improved from 0.41304 to 0.46739, saving model to model/crossval\model_crossval1.h5

Epoch 00005: val_accuracy improved from 0.46739 to 0.51087, saving model to model/crossval\model_crossval1.h5

Epoch 00006: val_accuracy did not improve from 0.51087

Epoch 00007: val_accuracy did not improve from 0.51087

Epoch 00008: val_accuracy improved from 0.51087 to 0.59239, saving model to model/crossval\model_crossval1.h5

Epoch 00009: val_accuracy did not improve from 0.59239

Epoch 00010: val_accuracy did not improve from 0.59239

Epoch 00011: val_accuracy did not improve from 0.59239

Epoch 00012: val_accuracy did not

In [5]:
print(f"the Averrage Accuracy score for 10 fold cross validation is {sum(accuracy)/10 *100} and recall is {sum(recall)/10*100}")

the Averrage Accuracy score for 10 fold cross validation is 76.9565224647522 and recall is 74.94636439955372
